In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

In [2]:
# Librerías de Scikit-learn para preprocesamiento y métricas
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc

In [3]:
# Imbalanced-learn para manejar desbalance de clases
from imblearn.over_sampling import RandomOverSampler

In [1]:
# Componentes de TensorFlow y Keras
import tensorflow as tf
from tensorflow.keras import layers, Model, callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50 # El modelo base que usaremos
from tensorflow.keras.applications.resnet import preprocess_input # Función de preprocesamiento específica para ResNet
from tensorflow.keras.metrics import AUC # Métrica de Área Bajo la Curva ROC

2025-05-28 14:22:18.157078: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748456538.265929    8196 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748456538.298553    8196 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748456538.518374    8196 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748456538.518414    8196 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748456538.518418    8196 computation_placer.cc:177] computation placer alr

In [2]:
import tensorflow as tf

print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available: 1


W0000 00:00:1748456545.942904    8196 gpu_device.cc:2430] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 5.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.


In [4]:
Model.__dict__

mappingproxy({'__module__': 'keras.src.models.model',
              '__doc__': 'A model grouping layers into an object with training/inference features.\n\n    There are three ways to instantiate a `Model`:\n\n    ## With the "Functional API"\n\n    You start from `Input`,\n    you chain layer calls to specify the model\'s forward pass,\n    and finally, you create your model from inputs and outputs:\n\n    ```python\n    inputs = keras.Input(shape=(37,))\n    x = keras.layers.Dense(32, activation="relu")(inputs)\n    outputs = keras.layers.Dense(5, activation="softmax")(x)\n    model = keras.Model(inputs=inputs, outputs=outputs)\n    ```\n\n    Note: Only dicts, lists, and tuples of input tensors are supported. Nested\n    inputs are not supported (e.g. lists of list or dicts of dict).\n\n    A new Functional API model can also be created by using the\n    intermediate tensors. This enables you to quickly extract sub-components\n    of the model.\n\n    Example:\n\n    ```python\n    

In [17]:
# Configuración para la visualización y logs de TensorFlow
sns.set_style("whitegrid") # Estilo para los gráficos de Seaborn
tf.get_logger().setLevel('ERROR') # Suprime mensajes informativos de TensorFlow, mostrando solo errores

In [ ]:
base_path = "images"  # Directorio raíz donde se encuentran las carpetas de categorías
categories = ["Healthy", "Tumor"] # Nombres de las subcarpetas y nuestras clas

In [ ]:
image_paths = []  # Lista para almacenar las rutas a cada imagen
labels = []       # Lista para almacenar la etiqueta de cada imagen

'images'

In [29]:
# Iterar sobre cada categoría definida
image_paths=[]
labels=[]
for category in categories:
    category_path = os.path.join(base_path, category) # Construir la ruta a la carpeta de la categoría
    if os.path.isdir(category_path):
        # Iterar sobre cada archivo de imagen dentro de la carpeta de la categoría
        for image_name in os.listdir(category_path):
            image_path = os.path.join(category_path, image_name) # Ruta completa a la imagen
            image_paths.append(image_path) # Añadir la ruta de la imagen a la lista
            labels.append(category)        # Añadir la etiqueta (nombre de la categoría) a la lista
    else:
        print(f"Advertencia: El directorio para la categoría '{category}' no fue encontrado en '{category_path}'")

# Crear un DataFrame de Pandas para almacenar las rutas de las imágenes y sus etiquetas
df = pd.DataFrame({"image_path": image_paths, "label": labels})

# Mostrar las primeras filas del DataFrame y la distribución de clases
print("DataFrame inicial con rutas de imágenes y etiquetas:")
print(df.head())
print("\nDistribución de clases inicial:")
print(df['label'].value_counts())

DataFrame inicial con rutas de imágenes y etiquetas:
                            image_path    label
0  images/Healthy/mri_healthy (17).jpg  Healthy
1  images/Healthy/mri_healthy (16).jpg  Healthy
2  images/Healthy/mri_healthy (36).jpg  Healthy
3   images/Healthy/mri_healthy (8).jpg  Healthy
4  images/Healthy/mri_healthy (11).jpg  Healthy

Distribución de clases inicial:
label
Healthy    53
Tumor      50
Name: count, dtype: int64


In [30]:
df

,image_path,label
0,images/Healthy/mri_healthy (17).jpg,Healthy
1,images/Healthy/mri_healthy (16).jpg,Healthy
2,images/Healthy/mri_healthy (36).jpg,Healthy
3,images/Healthy/mri_healthy (8).jpg,Healthy
4,images/Healthy/mri_healthy (11).jpg,Healthy
...,...,...
98,images/Tumor/glioma (30).jpg,Tumor
99,images/Tumor/glioma (48).jpg,Tumor
100,images/Tumor/glioma (27).jpg,Tumor
101,images/Tumor/glioma (31).jpg,Tumor
